In [19]:
using DataFrames
using LowRankModels

# Read in training data
df = readtable("../data/all_samples_ordinal_ready.csv", header=false)
all_data = sparse(Array(df[:, 3:end]))
m, n = size(all_data)
dropzeros!(all_data)

# Pull observed (nonzero) entries
i, j, v = findnz(all_data)
obs = collect(zip(i, j))

@show maximum(all_data), minimum(all_data), size(obs, 1), m, n

(maximum(all_data), minimum(all_data), size(obs, 1), m, n) = (5, 0, 2741198, 15655, 784)


(5, 0, 2741198, 15655, 784)

In [34]:
# construct the model
k = 15
#rx, ry = SimplexConstraint(), QuadReg(.01)
rx, ry = ZeroReg(), ZeroReg()

losses = Array{Loss}(n)
D = 0
for i=1:n
    options = unique(all_data[:, i])
    num_options = max(maximum(options), 3) # Min 3 levels to keep loss well-defined
    #num_options = maximum(options)
    losses[i] = BvSLoss(num_options)
    #losses[i] = MultinomialLoss(num_options)
    D += (num_options-1)
end
@show m, n, D


# Initialize X and Y
Xinit = randn(k, m)
Yord = randn(k, D)
yidxs = get_yidxs(losses)
for j=1:n
    prox!(ry, view(Yord,:,yidxs[j]), 1)
end

#glrm = GLRM(all_data, losses, rx, ry, k, obs=obs, scale=false, offset=true, X=Xinit, Y=Yord);
glrm = GLRM(all_data, QuadLoss(), rx, ry, k, obs=obs, scale=false, offset=true);



(m, n, D) = (15655, 784, 2013)


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Union{Int64, Range{Int64}}}, ::Int64) at ./deprecated.jl:57
 [3] get_yidxs(::Array{LowRankModels.Loss,1}) at /Users/kelley/.julia/v0.6/LowRankModels/src/losses.jl:77
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kelley/.julia/v0.6/IJulia/src/execute_request.jl:158
 [6] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:332
 [7] eventloop(::ZMQ.Socket) at /Users/kelley/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[34], in expression starting on line 22


In [ ]:
# fit full model
@time X,Y,ch = fit!(glrm, SparseProxGradParams(max_iter=500), verbose=true);


LowRankModels.SparseProxGradParams(1.0, 500, 1, 1.0e-5, 0.01)
Fitting GLRM


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Union{Int64, Range{Int64}}}, ::Int64) at ./deprecated.jl:57
 [3] get_yidxs(::Array{LowRankModels.Loss,1}) at /Users/kelley/.julia/v0.6/LowRankModels/src/losses.jl:77
 [4] (::LowRankModels.#kw##objective)(::Array{Any,1}, ::LowRankModels.#objective, ::LowRankModels.GLRM, ::Array{Float64,2}, ::Array{Float64,2}) at ./<missing>:0
 [5] #objective#61 at /Users/kelley/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:82 [inlined]
 [6] (::LowRankModels.#kw##objective)(::Array{Any,1}, ::LowRankModels.#objective, ::LowRankModels.GLRM) at ./<missing>:0
 [7] #fit!#81(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at /Users/kelley/.julia/v0.6/LowRankModels/src/algorithms/sparse_proxgrad.jl:50
 [8] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at ./<missing>:0
 [9] 

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] view(::Array{Float64,2}, ::Vararg{Any,N} where N) at ./deprecated.jl:31
 [3] #fit!#81(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at /Users/kelley/.julia/v0.6/LowRankModels/src/algorithms/sparse_proxgrad.jl:56
 [4] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at ./<missing>:0
 [5] include_string(::String, ::String) at ./loading.jl:522
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kelley/.julia/v0.6/IJulia/src/execute_request.jl:158
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/kelley/.julia/v0.6/Compat/src/Compat.jl:332
 [8] eventloop(::ZMQ.Socket) at /Users/kelley/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[36], in expression start

obj went up to 1.4146740996108274e12; reducing step size to 0.6666666666666666
obj went up to 1.7505143597952465e10; reducing step size to 0.4444444444444444
obj went up to 7.813997339110878e12; reducing step size to 0.3111111111111111
obj went up to 1.6961702759018042e11; reducing step size to 0.2074074074074074
obj went up to 2.0689912382116826e9; reducing step size to 0.1382716049382716
Iteration 10: objective value = 2.458651472984448e6
Iteration 20: objective value = 1.8497004072014927e6
Iteration 30: objective value = 1.6853509998969256e6
obj went up to 1.6342748230282767e6; reducing step size to 0.3984012724418721
obj went up to 1.6102419015594944e6; reducing step size to 0.2656008482945814
Iteration 40: objective value = 1.598704845670005e6
obj went up to 1.6003526970087264e6; reducing step size to 0.2152263274049104


In [29]:
# write to file
writecsv("/impute_bvs_simplex_no_sex_X$(k).csv", X)
writecsv("/impute_bvs_simplex_no_sex_Y$(k).csv", Y)
writecsv("/impute_bvs_simplex_no_sex_Z$(k).csv", impute(glrm))

LoadError: [91mUndefVarError: X not defined[39m